### Imports and Global Declarations

In [5]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
from wcs_client_adapter import WcsClientAdapter
from wcs_client_adapter import COLLECTION_TEXT_KEY, WCS_COLLECTION_NAME
from indexers import NaiveWcsIndexer
import csv
from typing import List
from typing import List, NamedTuple
import random


def display_md(content):
  display(Markdown(content))

### Index Paper for Retrieval

In [6]:
doc_uri = "https://arxiv.org/html/2312.10997v5"
indexer = NaiveWcsIndexer(doc_uri) # TODO: this calling syntax doesn't make it clear what side effects the constructor has

### Configure Language Model and Retrieval Model

In [7]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo")

wcs_client = WcsClientAdapter.get_wcs_client()
wcs_rm = WeaviateRM(WCS_COLLECTION_NAME, weaviate_client=wcs_client, weaviate_collection_text_key=COLLECTION_TEXT_KEY)
dspy.settings.configure(lm=turbo, rm=wcs_rm)

### Load Questions Dataset

In [8]:
answerable_questions_path = "./data/answerable-questions.csv"
unanswerable_questions_path = "./data/unanswerable-questions.csv"

def load_questions_from_csv(file_path: str) -> List[str]:
    questions = []
    with open(file_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            if row:
                questions.append(row[0])
    return questions

answerable_questions = load_questions_from_csv(answerable_questions_path)
unanswerable_questions = load_questions_from_csv(unanswerable_questions_path)
all_questions = answerable_questions + unanswerable_questions
all_qs_as_dspy_examples = trainset = [dspy.Example(question=question).with_inputs("question") for question in all_questions]

class DataSplits(NamedTuple):
    train: List
    dev: List
    test: List

def split_data(data: List, train_size: float, dev_size: float, test_size: float) -> DataSplits:
    if train_size + dev_size + test_size != 1:
        raise ValueError("The sum of train_size, dev_size, and test_size must be 1.")

    random.shuffle(data)  
    
    train_end = int(train_size * len(data))
    dev_end = train_end + int(dev_size * len(data))
    
    train_set = data[:train_end]
    dev_set = data[train_end:dev_end]
    test_set = data[dev_end:]
    
    return DataSplits(train=train_set, dev=dev_set, test=test_set)

splits = split_data(all_qs_as_dspy_examples, 0.7, 0.15, 0.15)

trainset = splits.train
devset = splits.dev
testset = splits.test

### Build Signatures

In [9]:
class GenerateAnswer(dspy.Signature):
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="between 1 and 4 sentences")
    
# TODO: LLM assessor signature goes here?

### Build Rag Pipeline as DSPy Module

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

### Optimize Pipeline

### Execute Pipeline

### Evaluate Complete RAG Pipeline

### Evaluate Retrieval Pipeline Stage